In [4]:
import os
import csv
from bs4 import BeautifulSoup

def parse_last_html():
    folder = "live_html"
    files = sorted(os.listdir(folder), key=lambda x: os.path.getmtime(os.path.join(folder, x)))
    last_file = os.path.join(folder, files[-1])
    
    print(f"📌 Reading File: {last_file}")
    
    with open(last_file, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f.read(), "html.parser")

    bid_list = soup.find("ul", id="biddinghistory")
    if not bid_list:
        print("⚠️ No Bidding History Found!")
        return

    items = bid_list.find_all("li")

    results = []
    current_lot = None
    bids = []
    status = ""

    for li in items:
        text = li.get_text(strip=True).lower()
        raw = li.get_text(strip=True)

        # 🆕 New Lot
        if "lot changed:" in text:
            if current_lot:
                last_bid = bids[0] if bids else ""
                results.append([current_lot, bids, status, last_bid])
            current_lot = raw.replace("Lot changed:", "").strip()
            bids = []
            status = ""

        # ❌ Not sold
        elif "not sold" in text:
            status = "not_sold"

        # 🟠 Provisionally sold
        elif "provisionally" in text and "sold" in text:
            status = "provisionally"
            price = raw.split("for")[-1].strip()
            bids.insert(0, price)

        # ✅ Sold final
        elif "sold" in text and "not sold" not in text:
            status = "sold"
            price = raw.split("for")[-1].strip()
            bids.insert(0, price)

        # ▶️ In progress
        elif "progress" in text:
            status = "in_progress"

        # 💰 Normal bid
        elif "bid:" in text:
            # only save bids if lot is sold or provisional
            if status in ["sold", "provisionally"]:
                price = raw.split("£")[-1].strip()
                bids.append("£" + price)

    # Last lot
    if current_lot:
        last_bid = bids[0] if bids else ""
        results.append([current_lot, bids, status, last_bid])

    # Print output
    print("\n=== BIDDING RESULT ===")
    for lot, prices, st, last_bid in results:
        print(f"{lot}  {prices}  {st}  Last Bid: {last_bid}")

    # Save CSV
    csv_file = "CCA_LIVE_data.csv"
    with open(csv_file, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["Lot", "Bids List", "Status", "Last Bid"])
        for row in results:
            writer.writerow(row)

    print(f"\n💾 CSV Saved Successfully: {csv_file}")


# RUN
parse_last_html()


📌 Reading File: live_html\39_SL68KBE.html

=== BIDDING RESULT ===
95  []  not_sold  Last Bid: 
94  ['£10,850', '£10,850', '£10,800', '£10,750', '£10,700', '£10,600', '£10,400', '£10,200', '£10,000']  provisionally  Last Bid: £10,850
93  ['£6,250', '£6,250', '£6,200', '£6,150', '£6,100', '£6,050', '£6,000', '£5,950', '£5,900', '£5,800', '£5,700', '£5,600', '£5,500']  provisionally  Last Bid: £6,250
92  ['£5,950', '£5,950', '£5,900', '£5,850', '£5,800', '£5,700', '£5,600', '£5,500', '£5,400']  provisionally  Last Bid: £5,950
91  ['£7,800', '£7,800', '£7,775', '£7,750', '£7,725', '£7,700', '£7,675', '£7,650', '£7,600', '£7,550', '£7,500', '£7,400', '£7,300', '£7,200', '£7,100', '£7,000']  provisionally  Last Bid: £7,800
90  ['£6,500', '£6,500', '£6,450', '£6,400', '£6,350', '£6,300', '£6,200', '£6,100', '£6,000', '£5,900', '£5,800']  sold  Last Bid: £6,500
89  ['£8,550', '£8,550', '£8,500', '£8,450', '£8,400', '£8,350', '£8,300', '£8,200', '£8,100', '£8,000', '£7,900', '£7,800']  provisio

In [5]:
import os
import csv
import pandas as pd
from bs4 import BeautifulSoup

csv_file = "CCA_LIVE_data.csv"
df = pd.read_csv(csv_file)

# Extract numeric lot number but DO NOT ADD column in CSV
df["_temp_lot"] = df["Lot"].str.extract(r'(\d+)').astype(int)

# Sort only for processing, but don't save the sorting column to CSV
df = df.sort_values(by="_temp_lot")

# Ensure Registration column exists
if "Registration" not in df.columns:
    df["Registration"] = ""

folder = "live_html"
html_files = sorted(os.listdir(folder), key=lambda x: os.path.getmtime(os.path.join(folder, x)))

def find_registration_in_html(lot_number):
    """Search Lot Registration inside all HTML"""
    for file in html_files:
        path = os.path.join(folder, file)
        with open(path, "r", encoding="utf-8") as f:
            soup = BeautifulSoup(f.read(), "html.parser")

        ul_tag = soup.find("ul", id="upcominglots")
        if not ul_tag:
            continue

        items = ul_tag.find_all("li")
        for li in items:
            strong_tag = li.find("strong")
            if strong_tag and f"Lot {lot_number}" in strong_tag.text:
                reg_tag = li.find("strong", string="• Registration:")
                if reg_tag:
                    reg = reg_tag.next_sibling.strip()
                    return reg
    return ""

for idx, row in df.iterrows():
    lot_num = row["_temp_lot"]
    reg = find_registration_in_html(lot_num)
    df.at[idx, "Registration"] = reg
    print(f"📌 Lot {lot_num} ⇒ Reg: {reg}")

# Drop temporary column before saving final CSV
df.drop(columns=["_temp_lot"], inplace=True)

df.to_csv(csv_file, index=False)
print(f"\n💾 CSV Updated Successfully with Registrations! → {csv_file}")


📌 Lot 1 ⇒ Reg: PXZ5984
📌 Lot 63 ⇒ Reg: SD65URB
📌 Lot 64 ⇒ Reg: ND69WHK
📌 Lot 64 ⇒ Reg: ND69WHK
📌 Lot 64 ⇒ Reg: ND69WHK
📌 Lot 65 ⇒ Reg: YK68FFA
📌 Lot 66 ⇒ Reg: KW17YHY
📌 Lot 67 ⇒ Reg: PN19YBF
📌 Lot 68 ⇒ Reg: NG18FNO
📌 Lot 69 ⇒ Reg: NK17PSZ
📌 Lot 70 ⇒ Reg: S80VOR
📌 Lot 71 ⇒ Reg: SH66ORO
📌 Lot 72 ⇒ Reg: PY68RBU
📌 Lot 73 ⇒ Reg: NU68LML
📌 Lot 74 ⇒ Reg: YS67VUT
📌 Lot 75 ⇒ Reg: MF69WXH
📌 Lot 75 ⇒ Reg: MF69WXH
📌 Lot 75 ⇒ Reg: MF69WXH
📌 Lot 76 ⇒ Reg: YK18NRS
📌 Lot 77 ⇒ Reg: SB17SHX
📌 Lot 78 ⇒ Reg: VE68UVD
📌 Lot 79 ⇒ Reg: PF69NZM
📌 Lot 80 ⇒ Reg: YD17GRV
📌 Lot 81 ⇒ Reg: NL67ZRT
📌 Lot 82 ⇒ Reg: SJ17BNN
📌 Lot 83 ⇒ Reg: DN67PWX
📌 Lot 84 ⇒ Reg: EJ18LUR
📌 Lot 85 ⇒ Reg: YH19OHD
📌 Lot 86 ⇒ Reg: SN18UYB
📌 Lot 87 ⇒ Reg: SF69VYX
📌 Lot 88 ⇒ Reg: SB69YGO
📌 Lot 89 ⇒ Reg: NL68MZZ
📌 Lot 90 ⇒ Reg: V666MRJ
📌 Lot 91 ⇒ Reg: NJ19LPZ
📌 Lot 92 ⇒ Reg: KW18TXR
📌 Lot 93 ⇒ Reg: NJ18CGX
📌 Lot 94 ⇒ Reg: YNZ2106
📌 Lot 95 ⇒ Reg: SL68KBE
📌 Lot 114 ⇒ Reg: 
📌 Lot 115 ⇒ Reg: 
📌 Lot 116 ⇒ Reg: 

💾 CSV Updated Successfully with Reg